# 1. Setup 
Run this first, it should crash the colab but its fine !

In [1]:
# run terminal comomands
!pip install -U -qq tensorflow==2.5.0
exit() # Runtime restart required!

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.4/454.4 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 39.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.4/462.4 KB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 16.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray 2022.12.0 requires numpy>=1.20, but you have numpy 1.19.5 which is incompatible.
xarray-einstats 0.4.0 requires numpy>=1.20, but you have numpy 1.19.5 which is incompatible.
pydantic 1.10.4 requires typing-extensions>=4.2.0, but you have typing-extension

# 2. Configuration 
Run this second, it will configure the model and make it ready to use

In [1]:
# import libs
import os
import sys
import json
import numpy as np
import time
from PIL import Image, ImageDraw

!git clone https://github.com/deedeeharris/Mask_RCNN

ROOT_DIR = 'Mask_RCNN'

sys.path.append(ROOT_DIR) 
from mrcnn.config import Config
import mrcnn.utils as utils
from mrcnn import visualize
import mrcnn.model as modellib

Cloning into 'Mask_RCNN'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 66 (delta 28), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (66/66), 465.52 KiB | 1.97 MiB/s, done.


In [2]:
class TrainConfig(Config):
    # Give the configuration a recognizable name
    NAME = "custom"

    # Train on 1 GPU and 1 image per GPU. Batch size is 1 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 5

    LEARNING_RATE = 0.001

    # Number of classes (including background) - IMPORTANT TO CHANGE ACCORDING TO YOUR LABELS IN YOUR JSON
    NUM_CLASSES = 1 + 2  # background + (red + green)

    # All of our training images are 1920x1012
    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 512
    
    # Matterport originally used resnet101, but I downsized to fit it on my graphics card
    BACKBONE = 'resnet50' # resnet50

    # To be honest, I haven't taken the time to figure out what these do
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    TRAIN_ROIS_PER_IMAGE = 32
    MAX_GT_INSTANCES = 50 
    POST_NMS_ROIS_INFERENCE = 500 
    POST_NMS_ROIS_TRAINING = 1000 
    
config = TrainConfig()

In [3]:
# download the trained model (red vs green flowers)
!pip install -U --no-cache-dir gdown --pre -qq

import gdown

file_id = '1fMYMGNEQstJ6WyagIyLWRmn-gyy6AsR5'
model_filename = 'mask_rcnn_redvsgreen_0030.h5'
gdown.download(f'https://drive.google.com/u/0/uc?id={file_id}', model_filename)



Downloading...
From: https://drive.google.com/u/0/uc?id=1fMYMGNEQstJ6WyagIyLWRmn-gyy6AsR5
To: /content/mask_rcnn_redvsgreen_0030.h5
100%|██████████| 185M/185M [00:02<00:00, 69.1MB/s]


'mask_rcnn_redvsgreen_0030.h5'

In [4]:
jsonTrain_file_id = '15dvrffNeDWe0LAe8EkY_R7l50khtAfqQ'
jsonTrain_filename = 'train.json'
gdown.download(f'https://drive.google.com/u/0/uc?id={jsonTrain_file_id}', jsonTrain_filename)

Downloading...
From: https://drive.google.com/u/0/uc?id=15dvrffNeDWe0LAe8EkY_R7l50khtAfqQ
To: /content/train.json
100%|██████████| 75.2k/75.2k [00:00<00:00, 63.7MB/s]


'train.json'

In [5]:
class InferenceConfig(TrainConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0.9 # CHANGE HERE IF YOU WANT

inference_config = InferenceConfig()

# Recreate the model in inference mode
test_model = modellib.MaskRCNN(
    mode="inference", 
    config=inference_config,
    model_dir='/content')

model_path = f'/content/{model_filename}'
print(model_path)

test_model.load_weights(model_path, by_name=True)

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Instructions for updating:
Use fn_output_signature instead


/content/mask_rcnn_redvsgreen_0030.h5


In [6]:
class CocoLikeDataset(utils.Dataset):
    """ Generates a COCO-like dataset, i.e. an image dataset annotated in the style of the COCO dataset.
        See http://cocodataset.org/#home for more information.
    """
    def load_data(self, annotation_json, images_dir):
        """ Load the coco-like dataset from json
        Args:
            annotation_json: The path to the coco annotations json file
            images_dir: The directory holding the images referred to by the json file
        """
        # Load json from file
        json_file = open(annotation_json)
        coco_json = json.load(json_file)
        json_file.close()
        
        # Add the class names using the base method from utils.Dataset
        source_name = "coco_like"
        for category in coco_json['categories']:
            class_id = category['id']
            class_name = category['name']
            if class_id < 1:
                print('Error: Class id for "{}" cannot be less than one. (0 is reserved for the background)'.format(class_name))
                return
            
            self.add_class(source_name, class_id, class_name)
        
        # Get all annotations
        annotations = {}
        for annotation in coco_json['annotations']:
            image_id = annotation['image_id']
            if image_id not in annotations:
                annotations[image_id] = []
            annotations[image_id].append(annotation)
        
        # Get all images and add them to the dataset
        seen_images = {}
        for image in coco_json['images']:
            image_id = image['id']
            if image_id in seen_images:
                print("Warning: Skipping duplicate image id: {}".format(image))
            else:
                seen_images[image_id] = image
                try:
                    image_file_name = image['file_name']
                    image_width = image['width']
                    image_height = image['height']
                except KeyError as key:
                    print("Warning: Skipping image (id: {}) with missing key: {}".format(image_id, key))
                
                image_path = os.path.abspath(os.path.join(images_dir, image_file_name))
                image_annotations = annotations[image_id]
                
                # Add the image using the base method from utils.Dataset
                self.add_image(
                    source=source_name,
                    image_id=image_id,
                    path=image_path,
                    width=image_width,
                    height=image_height,
                    annotations=image_annotations
                )
                
    def load_mask(self, image_id):
        """ Load instance masks for the given image.
        MaskRCNN expects masks in the form of a bitmap [height, width, instances].
        Args:
            image_id: The id of the image to load masks for
        Returns:
            masks: A bool array of shape [height, width, instance count] with
                one mask per instance.
            class_ids: a 1D array of class IDs of the instance masks.
        """
        image_info = self.image_info[image_id]
        annotations = image_info['annotations']
        instance_masks = []
        class_ids = []
        
        for annotation in annotations:
            class_id = annotation['category_id']
            mask = Image.new('1', (image_info['width'], image_info['height']))
            mask_draw = ImageDraw.ImageDraw(mask, '1')
            for segmentation in annotation['segmentation']:
                mask_draw.polygon(segmentation, fill=1)
                bool_array = np.array(mask) > 0
                instance_masks.append(bool_array)
                class_ids.append(class_id)

        mask = np.dstack(instance_masks)
        class_ids = np.array(class_ids, dtype=np.int32)
        
        return mask, class_ids

In [7]:
# get classnames
dataset_train = CocoLikeDataset()
dataset_train.load_data('/content/train.json', '/content')
dataset_train.prepare()
class_names = dataset_train.class_names

In [8]:
import skimage

mask_colors = [
    (0., 0., 0.), # Background
    (1., 0., 0.), # Red
    (0., 1., 0.)  # Green
]


def mask_image(img):
  img_arr = np.array(img)

  results = test_model.detect([img_arr], verbose=1)
  r = results[0]

  colors = tuple(np.take(mask_colors, r['class_ids'], axis=0))

  visualize.display_instances(img, r['rois'], r['masks'], r['class_ids'], 
                              class_names, r['scores'], figsize=(16, 8), 
                              colors=colors)
  return r

# 3. Upload image !
Upload your bell pepper image ! 
For better results, keep the picture simple with out many background objects.

Click on the start below ->

In [ ]:
#@title Upload image

# libs
from google.colab import files
from matplotlib import pyplot as plt
from skimage import io

# upload an image 
uploaded = files.upload()
img_path = list(uploaded.keys())[0]
img = io.imread(img_path)

# call our function
result = mask_image(img) 

# 4. Model Results
Run this after you upload your picture and get the visualisation

(results in the output below)

In [ ]:
totalFruits = len(result['class_ids'])
ripe = 0
for fruit in result['class_ids']:
  if fruit == 1:
    ripe+=1

if totalFruits == 0:
  print('Aint no fruits on this plant !')
else:
  precent=(ripe/totalFruits)*100
  print("Fruit number is "+str(totalFruits) + "\nRipe fruits precentage is: "+ str(int(precent))+'%')
